# Scoped Syntax Highlighting with Pygments, Jupyter, and FastHTML

This notebook shows how to:

* Get and use Pygments styles programmatically
* Extract and display the source code from Python functions
* Apply different Pygments syntax highlighting to different cells of the same notebook with proper CSS scoping
* Use Pygments-highlighted code in a FastHTML FastTag

In [131]:
from execnb.nbio import *
from fastcore.all import *
from fasthtml.common import *
from inspect import getsource
from IPython.display import display, HTML
import pygments
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter

## Pygments Styles

I was getting all Pygments styles the hard way in [my previous notebooks](https://github.com/audreyfeldroy/arg-blog-fasthtml/tree/main/nbs). There's a method for getting the highlight style names via Python:

In [59]:
styles = L(pygments.styles.get_all_styles())
print(styles)

['abap', 'algol', 'algol_nu', 'arduino', 'autumn', 'bw', 'borland', 'coffee', 'colorful', 'default', 'dracula', 'emacs', 'friendly_grayscale', 'friendly', 'fruity', 'github-dark', 'gruvbox-dark', 'gruvbox-light', 'igor', 'inkpot', 'lightbulb', 'lilypond', 'lovelace', 'manni', 'material', 'monokai', 'murphy', 'native', 'nord-darker', 'nord', 'one-dark', 'paraiso-dark', 'paraiso-light', 'pastie', 'perldoc', 'rainbow_dash', 'rrt', 'sas', 'solarized-dark', 'solarized-light', 'staroffice', 'stata-dark', 'stata-light', 'tango', 'trac', 'vim', 'vs', 'xcode', 'zenburn']


## Inspect and `getsource`

Let's grab a function to highlight. How about `read_nb` from execnb:

In [66]:
rn = getsource(read_nb)
rn

'def read_nb(path):\n    "Return notebook at `path`"\n    res = dict2nb(_read_json(path, encoding=\'utf-8\'))\n    res[\'path_\'] = str(path)\n    return res\n'

We have to print it to see it nicely:

In [64]:
print(rn)

def read_nb(path):
    "Return notebook at `path`"
    res = dict2nb(_read_json(path, encoding='utf-8'))
    res['path_'] = str(path)
    return res



## Pygments `highlight`

As in previous posts, we call `highlight` to highlight a Python code block like this:

In [68]:
h = highlight(rn, PythonLexer(), HtmlFormatter(style='tango'))
h

'<div class="highlight"><pre><span></span><span class="k">def</span> <span class="nf">read_nb</span><span class="p">(</span><span class="n">path</span><span class="p">):</span>\n    <span class="s2">&quot;Return notebook at `path`&quot;</span>\n    <span class="n">res</span> <span class="o">=</span> <span class="n">dict2nb</span><span class="p">(</span><span class="n">_read_json</span><span class="p">(</span><span class="n">path</span><span class="p">,</span> <span class="n">encoding</span><span class="o">=</span><span class="s1">&#39;utf-8&#39;</span><span class="p">))</span>\n    <span class="n">res</span><span class="p">[</span><span class="s1">&#39;path_&#39;</span><span class="p">]</span> <span class="o">=</span> <span class="nb">str</span><span class="p">(</span><span class="n">path</span><span class="p">)</span>\n    <span class="k">return</span> <span class="n">res</span>\n</pre></div>\n'

Then to display that in a notebook:

In [69]:
HTML(h)

## Highlighting Code In-Notebook

Putting `highlight` and `HTML` into a function together, building up from above:

In [71]:
def show(c): return HTML(highlight(c, PythonLexer(), HtmlFormatter(style='tango')))

In [73]:
show(rn)

## Function Getting Its Own Source

To get some source code to highlight without having to read a notebook:

In [88]:
def get_myself(): return getsource(get_myself)

In [89]:
get_myself()

'def get_myself(): return getsource(get_myself)\n'

## Function Highlighting Itself

Putting together `highlight`, `HTML`, and `getsource`:

In [96]:
def show(c=None): 
    if not c: c = getsource(show)
    return HTML(highlight(c, PythonLexer(), HtmlFormatter(style='tango')))

In [97]:
show()

## Adding a `style` Arg

I wanted to show my code with a particular Pygments style:

In [99]:
def show(c=None, style='tango'): 
    if not c: c = getsource(show)
    return HTML(highlight(c, PythonLexer(), HtmlFormatter(style=style)))

In [102]:
show(style='zenburn')

Something's not right here. That showed no colors.

## Understanding Pygments Style Defs

In Pygments, style defs are CSS style definitions:

In [108]:
sd = HtmlFormatter(style='zenburn').get_style_defs()
sd[:200]

'pre { line-height: 125%; }\ntd.linenos .normal { color: #5d6262; background-color: #353535; padding-left: 5px; padding-right: 5px; }\nspan.linenos { color: #5d6262; background-color: #353535; padding-le'

In [116]:
s = L(sd.splitlines())
s

(#84) ['pre { line-height: 125%; }','td.linenos .normal { color: #5d6262; background-color: #353535; padding-left: 5px; padding-right: 5px; }','span.linenos { color: #5d6262; background-color: #353535; padding-left: 5px; padding-right: 5px; }','td.linenos .special { color: #7a8080; background-color: #353535; padding-left: 5px; padding-right: 5px; }','span.linenos.special { color: #7a8080; background-color: #353535; padding-left: 5px; padding-right: 5px; }','.hll { background-color: #484848 }','.c { color: #7f9f7f; font-style: italic } /* Comment */','.err { color: #e37170; font-weight: bold } /* Error */','.esc { color: #dcdccc } /* Escape */','.g { color: #ecbcbc; font-weight: bold } /* Generic */','.k { color: #efdcbc } /* Keyword */','.l { color: #9fafaf } /* Literal */','.n { color: #dcdccc } /* Name */','.o { color: #f0efd0 } /* Operator */','.x { color: #dcdccc } /* Other */','.p { color: #f0efd0 } /* Punctuation */','.ch { color: #7f9f7f; font-style: italic } /* Comment.Hashbang

In [117]:
s[0]

'pre { line-height: 125%; }'

In [123]:
s[6]

'.c { color: #7f9f7f; font-style: italic } /* Comment */'

In [127]:
s[10]

'.k { color: #efdcbc } /* Keyword */'

## Looking at Hex Colors With FastTags

In style `zenburn`, comments are colored in `#7f9f7f`. Let's see what this looks like with a `Div` FastTag:

In [134]:
cdiv = Div('#7f9f7f', style="background-color:#7f9f7f;")
cdiv

```html
<div style="background-color:#7f9f7f;">#7f9f7f</div>

```

In [143]:
HTML(to_xml(cdiv))

In [144]:
def show_color(c): return HTML(to_xml(Div(c, style=f"background-color:{c};")))

Keywords in `zenburn` are colored with `#efdcbc`:

In [142]:
show_color("#efdcbc")

## Pygments Styles in FastHTML FastTags

Putting `zenburn` comment and keyword styles in a `Style` FastTag:

In [151]:
Style(s[6], s[10])

```html
<style>
.c { color: #7f9f7f; font-style: italic } /* Comment */.k { color: #efdcbc } /* Keyword */</style>

```

## Pygments Highlighting in FastTags

Recall Pygments `highlight` from earlier generates a `div` containing `pre` full of `span` tags: 

In [155]:
h = highlight(rn, PythonLexer(), HtmlFormatter(style='tango'))
print(h)

<div class="highlight"><pre><span></span><span class="k">def</span> <span class="nf">read_nb</span><span class="p">(</span><span class="n">path</span><span class="p">):</span>
    <span class="s2">&quot;Return notebook at `path`&quot;</span>
    <span class="n">res</span> <span class="o">=</span> <span class="n">dict2nb</span><span class="p">(</span><span class="n">_read_json</span><span class="p">(</span><span class="n">path</span><span class="p">,</span> <span class="n">encoding</span><span class="o">=</span><span class="s1">&#39;utf-8&#39;</span><span class="p">))</span>
    <span class="n">res</span><span class="p">[</span><span class="s1">&#39;path_&#39;</span><span class="p">]</span> <span class="o">=</span> <span class="nb">str</span><span class="p">(</span><span class="n">path</span><span class="p">)</span>
    <span class="k">return</span> <span class="n">res</span>
</pre></div>



This is a nice string of HTML to use with FastTags. I use `NotStr` to make it work well with a `Div` FastTag:

In [157]:
Div(NotStr(h), id="container")

```html
<div id="container"><div class="highlight"><pre><span></span><span class="k">def</span> <span class="nf">read_nb</span><span class="p">(</span><span class="n">path</span><span class="p">):</span>
    <span class="s2">&quot;Return notebook at `path`&quot;</span>
    <span class="n">res</span> <span class="o">=</span> <span class="n">dict2nb</span><span class="p">(</span><span class="n">_read_json</span><span class="p">(</span><span class="n">path</span><span class="p">,</span> <span class="n">encoding</span><span class="o">=</span><span class="s1">&#39;utf-8&#39;</span><span class="p">))</span>
    <span class="n">res</span><span class="p">[</span><span class="s1">&#39;path_&#39;</span><span class="p">]</span> <span class="o">=</span> <span class="nb">str</span><span class="p">(</span><span class="n">path</span><span class="p">)</span>
    <span class="k">return</span> <span class="n">res</span>
</pre></div>
</div>

```

Adding style:

In [160]:
styled_container = Div(Style(s[6], s[10]), NotStr(h), id="container")
styled_container

```html
<div id="container">
  <style>
.c { color: #7f9f7f; font-style: italic } /* Comment */.k { color: #efdcbc } /* Keyword */  </style>
<div class="highlight"><pre><span></span><span class="k">def</span> <span class="nf">read_nb</span><span class="p">(</span><span class="n">path</span><span class="p">):</span>
    <span class="s2">&quot;Return notebook at `path`&quot;</span>
    <span class="n">res</span> <span class="o">=</span> <span class="n">dict2nb</span><span class="p">(</span><span class="n">_read_json</span><span class="p">(</span><span class="n">path</span><span class="p">,</span> <span class="n">encoding</span><span class="o">=</span><span class="s1">&#39;utf-8&#39;</span><span class="p">))</span>
    <span class="n">res</span><span class="p">[</span><span class="s1">&#39;path_&#39;</span><span class="p">]</span> <span class="o">=</span> <span class="nb">str</span><span class="p">(</span><span class="n">path</span><span class="p">)</span>
    <span class="k">return</span> <span class="n">res</span>
</pre></div>
</div>

```

To display it in-notebook here:

In [161]:
HTML(to_xml(styled_container))

## Pygments Background Color

The Pygments [`get_style_defs` docs](https://pygments.org/docs/api/#pygments.formatter.Formatter.get_style_defs) say you can specify a CSS selector to prefix styles with:

In [185]:
sd = HtmlFormatter(style='zenburn').get_style_defs('.highlight')
sd[:500]

'pre { line-height: 125%; }\ntd.linenos .normal { color: #5d6262; background-color: #353535; padding-left: 5px; padding-right: 5px; }\nspan.linenos { color: #5d6262; background-color: #353535; padding-left: 5px; padding-right: 5px; }\ntd.linenos .special { color: #7a8080; background-color: #353535; padding-left: 5px; padding-right: 5px; }\nspan.linenos.special { color: #7a8080; background-color: #353535; padding-left: 5px; padding-right: 5px; }\n.highlight .hll { background-color: #484848 }\n.highlight'

I see all the zenburn style defs with background colors are early on:

In [187]:
Style(sd[:600])

```html
<style>pre { line-height: 125%; }
td.linenos .normal { color: #5d6262; background-color: #353535; padding-left: 5px; padding-right: 5px; }
span.linenos { color: #5d6262; background-color: #353535; padding-left: 5px; padding-right: 5px; }
td.linenos .special { color: #7a8080; background-color: #353535; padding-left: 5px; padding-right: 5px; }
span.linenos.special { color: #7a8080; background-color: #353535; padding-left: 5px; padding-right: 5px; }
.highlight .hll { background-color: #484848 }
.highlight { background: #3f3f3f; color: #dcdccc }
.highlight .c { color: #7f9f7f; font-style: italic } /* Com</style>

```

In [178]:
show_color("#484848")

In [179]:
show_color("#353535")

In [194]:
styled_container = Div(Style(sd), NotStr(h), id="container")
HTML(to_xml(styled_container))

## Combining Everything Into `show`

Let's combine everything we've learned into a function:

In [199]:
def show(c=None, style='monokai'): 
    if not c: c = getsource(show)
    fm = HtmlFormatter(style=style)
    h = highlight(c, PythonLexer(), fm)
    sd = fm.get_style_defs('.highlight')
    styled_container = Div(Style(sd), NotStr(h), id="container")
    return HTML(to_xml(styled_container))

In [204]:
show(style='monokai')

In [205]:
show(style='lightbulb')

## Fixing CSS Scope Leakage

Let's see if we can customize the `highlight` class

In [209]:
fm = HtmlFormatter(style='monokai')
h = highlight("print('Hi')", PythonLexer(), fm)
h

'<div class="highlight"><pre><span></span><span class="nb">print</span><span class="p">(</span><span class="s1">&#39;Hi&#39;</span><span class="p">)</span>\n</pre></div>\n'

In [211]:
def show(c=None, style='monokai'): 
    if not c: c = getsource(show)
    fm = HtmlFormatter(style=style)
    h = highlight(c, PythonLexer(), fm)
    sd = fm.get_style_defs(f'#{style}')
    styled_container = Div(Style(sd), NotStr(h), id=style)
    return HTML(to_xml(styled_container))

In [213]:
show(style='monokai')

In [214]:
show(style='lightbulb')

The above 2 appeared to work correctly, but this didn't, so something's wrong:

In [215]:
show(style='paraiso-light')

In [221]:
print(HtmlFormatter(style='paraiso-light').get_style_defs('#paraiso-light')[:1000])

pre { line-height: 125%; }
td.linenos .normal { color: inherit; background-color: transparent; padding-left: 5px; padding-right: 5px; }
span.linenos { color: inherit; background-color: transparent; padding-left: 5px; padding-right: 5px; }
td.linenos .special { color: #000000; background-color: #ffffc0; padding-left: 5px; padding-right: 5px; }
span.linenos.special { color: #000000; background-color: #ffffc0; padding-left: 5px; padding-right: 5px; }
#paraiso-light .hll { background-color: #a39e9b }
#paraiso-light { background: #e7e9db; color: #2f1e2e }
#paraiso-light .c { color: #8d8687 } /* Comment */
#paraiso-light .err { color: #ef6155 } /* Error */
#paraiso-light .k { color: #815ba4 } /* Keyword */
#paraiso-light .l { color: #f99b15 } /* Literal */
#paraiso-light .n { color: #2f1e2e } /* Name */
#paraiso-light .o { color: #5bc4bf } /* Operator */
#paraiso-light .p { color: #2f1e2e } /* Punctuation */
#paraiso-light .ch { color: #8d8687 } /* Comment.Hashbang */
#paraiso-light .cm { co

The background color is supposed to be:

In [223]:
show_color("#a39e9b")

I think `get_style_defs('#paraiso-light')` where that ID is on the parent div is too hacky here. I feel like `<div class="highlight">` itself should get the ID.

In [224]:
print(HtmlFormatter(style='paraiso-light').get_background_style_defs('#paraiso-light')[:1000])

['#paraiso-light .hll { background-color: #a39e9b }', '#paraiso-light { background: #e7e9db; color: #2f1e2e }']


In [226]:
c = 'print("Hi")'
fm = HtmlFormatter(style='paraiso-light', cssclass='audrey')
h = highlight(c, PythonLexer(), fm)
h

'<div class="audrey"><pre><span></span><span class="nb">print</span><span class="p">(</span><span class="s2">&quot;Hi&quot;</span><span class="p">)</span>\n</pre></div>\n'

In [241]:
def show(c=None, style='monokai'): 
    if not c: c = getsource(show)
    fm = HtmlFormatter(style=style, cssclass=style)
    h = highlight(c, PythonLexer(), fm)
    sd = fm.get_style_defs(f".{style}")
    styled_container = Div(Style(sd), NotStr(h), id=style)
    return HTML(to_xml(styled_container))

In [242]:
show(style='paraiso-light')

In [243]:
show(c="print('Hey')", style="dracula")

In [244]:
show(style="dracula")

In [245]:
show(style="gruvbox-dark")

In [246]:
show(style="solarized-dark")

Success! The cells above are syntax-highlighted without their CSS interfering with each other.

## Summary

I've created a function for displaying Pygments syntax-highlighted code in Jupyter notebooks with properly-scoped CSS. To do this, I discovered I could:

1. Use Pygments' `HtmlFormatter`'s `cssclass` parameter to change the name of the outer `highlight` div to the Pygments style name.
2. Use `get_style_defs` to scope style definitions to that name, to prevent CSS conflicts
3. Combine it into a tiny `show` function for use in future notebooks

In [ ]:
def show(c=None, style='monokai'): 
    if not c: c = getsource(show)
    fm = HtmlFormatter(style=style, cssclass=style)
    h = highlight(c, PythonLexer(), fm)
    sd = fm.get_style_defs(f".{style}")
    styled_container = Div(Style(sd), NotStr(h), id=style)
    return HTML(to_xml(styled_container))

You can use this to show code blocks in Jupyter notebooks, allowing different Pygments syntax highlighting themes in the same notebook. All without CSS leaking between Pygments styles.